In [ ]:
## Flow

1) Get number of pages
2) Parallelise job, making one driver for each page.
3) # Do I want to do this in a massive for loop? Not really..But I need to somehow change the page number for each page

- Remember: USP == sORFs

In [12]:
## *** The workflow *** ## 
## Built to be flexible and scalable (test with different organisms etc.)
## First import modules
## Maybe make virtualenv with all the necessary dependencies already installed? - would probs help with later bedtools stuff etc


import httplib2
from bs4 import BeautifulSoup, SoupStrainer
from bs4.element import Comment
import urllib
import re
from ast import literal_eval
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
import timeit



In [11]:
## Function definitions

## HTML parsing

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



In [6]:
tp = ThreadPoolExecutor(max_workers=5)
help(tp.submit)

Help on method submit in module concurrent.futures.thread:

submit(fn, *args, **kwargs) method of concurrent.futures.thread.ThreadPoolExecutor instance
    Submits a callable to be executed with the given arguments.
    
    Schedules the callable to be executed as fn(*args, **kwargs) and returns
    a Future instance representing the execution of the callable.
    
    Returns:
        A Future representing the given call.



# Oct 4th, 2017

In [16]:
## Need to give this function the URL of each page, and get it to parse each page, find all links
## then get all chrom_coords for each page

def get_page_deets(page_URL):

    ## First get all links on one page
    #response = urllib.request.urlopen(URL)
#     http = httplib2.Http()
#     status, response = http.request(page_URL)

    response = s.get(page_URL).text

    all_links = []
    for link in BeautifulSoup(response, "lxml", parse_only=SoupStrainer('a')):
        try:
            all_links.append(link['href'])
        except:
            pass
    ## Filter out other irrelevant links in the page
    SM_prot_codes = [u for u in all_links if 'SmProt.php' in u]



    IDs = []
    Sequence = []
    Length = []
    Transcript_ID = []
    Gene_ID = []
    Location = []
    Cell_Lines = []
    Data_Source = []
    Experiment = []
    Experimental_Description = []

  
    for SM_prot_code in SM_prot_codes:
        URL = 'http://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_code
        # Think I'll use http.request as faster 
        #response = urllib.request.urlopen(URL)
#       status, response = http.request(URL)
        each_SEP = s.get(URL)
        

        ## Beaut Soup the response
        text = text_from_html(each_SEP.text)

        ## Regexp extract relevant details for table
        ID = re.findall(r'Small Protein ID (.*?) Sequence',text)
        sequence = re.findall(r'Sequence (.*?) Small Protein Length',text)
        length = re.findall(r'Small Protein Length (.*?) ORF Type',text)
        transcript_ID = re.findall(r'sORF Transcript ID (.*?) Gene Symbol',text)
        gene_ID = re.findall(r'Gene Symbol (.*?) Gene Synonyms',text)
        location = re.findall(r'Location (.*?) Cell lines',text)
        cell_lines = re.findall(r'Cell lines Or Tissues (.*?) Data Source',text)
        data_source = re.findall(r'Data Source (.*?) Detailed information',text)
        experiment = re.findall(r'Experiment (.*?) Description',text)
        experimental_description = re.findall(r'Description (.*?) References',text)
 

        # Create array for dataframe
        IDs.append(ID)
        Sequence.append(sequence)
        Length.append(length)
        Transcript_ID.append(transcript_ID)
        Gene_ID.append(gene_ID)
        Location.append(location)
        Cell_Lines.append(cell_lines)
        Data_Source.append(data_source)
        Experiment.append(experiment)
        Experimental_Description.append(experimental_description)
       
        
    current_df = pd.DataFrame([])
    current_df['IDs'] = IDs
    current_df['Sequence'] = Sequence
    current_df['Length'] = Length
    current_df['Transcript_ID'] = Transcript_ID
    current_df['Gene_ID'] = Gene_ID
    current_df['Location'] = Location
    current_df['Cell_Lines'] = Cell_Lines
    current_df['Data_Source'] = Data_Source
    current_df['Experiment'] = Experiment
    current_df['Experimental_Description'] = Experimental_Description
    
    
    current_df.to_csv('alldata_page%s' %j)
    
    

In [17]:
s = requests.Session()
start_time = timeit.default_timer()
tp = ThreadPoolExecutor(max_workers=total_pages)
s = requests.Session()

for j in range(0,total_pages):

    results_per_page = 10000
    orf_type = 'sORF'
    gene_type = 'All'
    function ='All'
    organism = 'human'
    datasource = 'All'
    page_no = j+1

    ## Homepage URL
    page_URL = "http://bioinfo.ibp.ac.cn/SmProt/browse.php?page=%s&perpage=%s&orfType=%s&geneType=%s&function=%s&org=%s&datasource=%s" %(page_no,results_per_page,orf_type,gene_type,function,organism,datasource)#
     
    tp.submit(get_page_deets, page_URL)
    
    
tp.shutdown(wait=True)    
elapsed = timeit.default_timer() - start_time

In [19]:
elapsed

4026.8001635569963

In [20]:
current_df

NameError: name 'current_df' is not defined

In [22]:
## This gets all links on one page ## 
def get_all_links(page_URL):

    response = s.get(page_URL).text
    
    ## Slower alternatives..
    # response = urllib.request.urlopen(URL)
    # http = httplib2.Http()
    # status, response = http.request(URL)

    # all_links = []
    # for link in BeautifulSoup(response, parseOnlyThese=SoupStrainer('a')):
    #     if link.hasattr('href'):
    #         all_links.append(link['href'])


    all_links = []
    for link in BeautifulSoup(response, "lxml", parse_only=SoupStrainer('a')):
        try:
            all_links.append(link['href'])
        except:
            pass
    ## Filter out other irrelevant links in the page
    SM_prot_codes = [u for u in all_links if 'SmProt.php' in u]
    #global SM_prot_codes

In [27]:
# test this parallel thread thing - time it as well

start_time = timeit.default_timer()
n_threads = 10 
#tp = ThreadPoolExecutor(max_workers=n_threads)
s = requests.Session()

for i in range(0,10):

    results_per_page = 2000
    orf_type = 'sORF'
    gene_type = 'All'
    function ='All'
    organism = 'human'
    datasource = 'All'
    page_no = i

    page_URL = "http://bioinfo.ibp.ac.cn/SmProt/browse.php?page=%s&perpage=%s&orfType=%s&geneType=%s&function=%s&org=%s&datasource=%s" %(page_no,results_per_page,orf_type,gene_type,function,organism,datasource)#
    get_all_links(page_URL)
    #tp.submit(get_all_links, page_URL)

    
#tp.shutdown(wait=True)    
elapsed = timeit.default_timer() - start_time

In [26]:
elapsed

8.024742046996835

In [28]:
elapsed

13.294918738989509

In [1]:
# Need to wrap up the scraping of a whole page into one function
# Make a function that does the following:

# - Each page contains a list of sORFs.
# For each page:
#               - Go onto that page and get all links to sORF data.
#               - Click on each link on that page, parse details and store somewhere


# for i in range(0,total_pages):
    # could have new function definition in each iteration?? - no need, just have iteration as an input 
    # into the function. Wait do I even need to do that? Can just define the Page URL outside the function,
    # then give the page URL as the input to the function
    


NameError: name 'total_pages' is not defined

In [10]:
# Define page settings - set all answers as strings 
# ^^ set these as widgets before pushing to github (http://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)

results_per_page = 10000
orf_type = 'sORF'
gene_type = 'All'
function ='All'
organism = 'human'
datasource = 'All'

## Homepage URL
URL = "http://bioinfo.ibp.ac.cn/SmProt/browse.php?page=1&perpage=%s&orfType=%s&geneType=%s&function=%s&org=%s&datasource=%s" %(results_per_page,orf_type,gene_type,function,organism,datasource)#

## Find number of pages - there are different request libraries to do this - I want the fastest one

# response = urllib.request.urlopen(URL)
## ***** Start session here ***** ##
s = requests.Session()
response = s.get(URL)
## Beaut Soup the response
text = text_from_html(response.text)

total_pages_str = re.findall(r'Total Page: (.*?) Total amount:',text)
total_pages = literal_eval(total_pages_str[0])

## I want to run the scraping function in parallel over each page, so as many parallel functions 


In [ ]:
## This gets all links on one page ## 
def get_all_links(page_URL):

    response = s.get(page_URL).text
    
    ## Slower alternatives..
    # response = urllib.request.urlopen(URL)
    # http = httplib2.Http()
    # status, response = http.request(URL)

    # all_links = []
    # for link in BeautifulSoup(response, parseOnlyThese=SoupStrainer('a')):
    #     if link.hasattr('href'):
    #         all_links.append(link['href'])


    all_links = []
    for link in BeautifulSoup(response, "lxml", parse_only=SoupStrainer('a')):
        try:
            all_links.append(link['href'])
        except:
            pass
    ## Filter out other irrelevant links in the page
    SM_prot_codes = [u for u in all_links if 'SmProt.php' in u]
    #global SM_prot_codes

In [ ]:
# Try setting up one thread per page
n_threads = total_pages
## Parallises the job - divides date range by no. of threads specified probs
tp = ThreadPoolExecutor(max_workers=n_threads)
# for i in range(0, n_days):
#     since_query = self.since + datetime.timedelta(days=i)
#     until_query = self.since + datetime.timedelta(days=(i + 1))
#     day_query = "%s since:%s until:%s" % (query, since_query.strftime("%Y-%m-%d"),
#                                           until_query.strftime("%Y-%m-%d"))
    tp.submit(self.perform_search, day_query)
tp.shutdown(wait=True)

In [6]:
n_threads=82
tp = ThreadPoolExecutor(max_workers=n_threads)
tp.submit(get_all_links,URL)

<Future at 0x7efec0346438 state=running>

In [36]:
n_threads = total_pages
tp = ThreadPoolExecutor(max_workers=n_threads)
for x in range(0,n_threads):
    page_no = x
    page_URL
    tp.submit()

## Need to wrap the below code into a function - called extract all

TypeError: submit() missing 1 required positional argument: 'fn'

In [ ]:
## code below should take < 20 mins.. we'll see..
## Should also benchmark requests.Session().get request - uses same underlying TCP connection so apparently much faster
## Also need to add a timeout to requests to stop everything being slowed by one slow request

## Example of parallelising (saved lots of time here: http://elliothallmark.com/2016/12/23/requests-with-concurrent-futures-in-python-2-7/)
# import requests
# from concurrent.futures import ThreadPoolExecutor, wait, as_completed
# from time import time
# urls = ['google.com','cnn.com','reddit.com','imgur.com','yahoo.com']
# urls = ["http://"+url for url in urls]
# # Time requests running synchronously
# then = time()
# sync_results = map(requests.get, urls)
# print "Synchronous done in %s" % (time()-then)
# # Time requests running in threads
# then = time()
# pool = ThreadPoolExecutor(len(urls))  # for many urls, this should probably be capped at some value.

# futures = [pool.submit(requests.get,url) for url in urls]
# results = [r.result() for r in as_completed(futures)]
# print "Threadpool done in %s" % (time()-then)


In [1]:
URL = 'https://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_codes[0]
http = httplib2.Http()
status, response = http.request(URL)


NameError: name 'SM_prot_codes' is not defined

In [33]:
## Need to give this function the URL of each page, and get it to parse each page, find all links
## then get all chrom_coords for each page

def get_page_deets(page_URL)

    ## First get all links on one page
    #response = urllib.request.urlopen(URL)
#     http = httplib2.Http()
#     status, response = http.request(page_URL)

    response = s.get(page_URL).text

    all_links = []
    for link in BeautifulSoup(response, "lxml", parse_only=SoupStrainer('a')):
        try:
            all_links.append(link['href'])
        except:
            pass
    ## Filter out other irrelevant links in the page
    SM_prot_codes = [u for u in all_links if 'SmProt.php' in u]



    IDs = []
    Sequence = []
    Length = []
    Transcript_ID = []
    Gene_ID = []
    Location = []
    Cell_Lines = []
    Data_Source = []
    Experiment = []
    Experimental_Description = []

    http = httplib2.Http()
#     start_time = timeit.default_timer()

    i = 0
    for SM_prot_code in SM_prot_codes:
        URL = 'http://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_code
        # Think I'll use http.request as faster 
        #response = urllib.request.urlopen(URL)
#       status, response = http.request(URL)
        each_SEP = s.get(URL)
        

        ## Beaut Soup the response
        text = text_from_html(each_SEP)

        ## Regexp extract relevant details for table
        ID = re.findall(r'Small Protein ID (.*?) Sequence',text)
        sequence = re.findall(r'Sequence (.*?) Small Protein Length',text)
        length = re.findall(r'Small Protein Length (.*?) ORF Type',text)
        transcript_ID = re.findall(r'sORF Transcript ID (.*?) Gene Symbol',text)
        gene_ID = re.findall(r'Gene Symbol (.*?) Gene Synonyms',text)
        location = re.findall(r'Location (.*?) Cell lines',text)
        cell_lines = re.findall(r'Cell lines Or Tissues (.*?) Data Source',text)
        data_source = re.findall(r'Data Source (.*?) Detailed information',text)
        experiment = re.findall(r'Experiment (.*?) Description',text)
        experimental_description = re.findall(r'Description (.*?) References',text)
        ## Below probably present depending on Data Source (create if loop maybe)
    #     PubMed_ref = re.findall(r'PubMed ID (.*?) Title',text)
    #     Paper_Title = re.findall(r'Title (.*?) Authors',text)

        # Create array for dataframe
        IDs.append(ID)
        Sequence.append(sequence)
        Length.append(length)
        Transcript_ID.append(transcript_ID)
        Gene_ID.append(gene_ID)
        Location.append(location)
        Cell_Lines.append(cell_lines)
        Data_Source.append(data_source)
        Experiment.append(experiment)
        Experimental_Description.append(experimental_description)
        i += 1

        if i % 100 == 0:
            print(i)
        
    current_df = pd.DataFrame([])
    current_df['IDs'] = IDs
    current_df['Sequence'] = Sequence
    current_df['Length'] = Length
    current_df['Transcript_ID'] = Transcript_ID
    current_df['Gene_ID'] = Gene_ID
    current_df['Location'] = Location
    current_df['Cell_Lines'] = Cell_Lines
    current_df['Data_Source'] = Data_Source
    current_df['Experiment'] = Experiment
    current_df['Experimental_Description'] = Experimental_Description
    
    # Get database number
    #db_number = re.findall(r'')
    current_df.to_csv('all_data_page%s' %j)
    
    
#     time_elapsed = timeit.default_timer() - start_time
    ## Get all links (number just increases by 1 each time), then click on each and webscrape chrom coords.

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [8]:
s.get(URL)

<Response [200]>

In [7]:
s = requests.Session()

start_time = timeit.default_timer()

for SM_prot_code in SM_prot_codes[0:10]:
    URL = 'http://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_code
    response = s.get(URL)
    ## Beaut Soup the response
    #text = text_from_html(response)
    
elapsed = timeit.default_timer() - start_time
print(elapsed)

NameError: name 'timeit' is not defined

In [ ]:
## Notes

# below code shows http.request is ~2x faster than urllib requests or requests
# compare http, requests, and urllib with timeit
import timeit
# timeit.timeit(urllib.request.urlopen(URL))

start_time = timeit.default_timer()

for SM_prot_code in SM_prot_codes[0:10]:
    URL = 'http://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_code
    response = urllib.request.urlopen(URL)
    ## Beaut Soup the response
    text = text_from_html(response)
    
elapsed = timeit.default_timer() - start_time
print(elapsed)

# compare http, requests, and urllib with timeit
import timeit
import requests
# timeit.timeit(urllib.request.urlopen(URL))

start_time = timeit.default_timer()

for SM_prot_code in SM_prot_codes[0:10]:
    URL = 'http://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_code
    response = requests.get(URL)
    ## Beaut Soup the response
    #text = text_from_html(response)
    
elapsed = timeit.default_timer() - start_time
print(elapsed)


# compare http, requests, and urllib with timeit
import timeit
import requests
import httplib2
# timeit.timeit(urllib.request.urlopen(URL))
http = httplib2.Http()
start_time = timeit.default_timer()

for SM_prot_code in SM_prot_codes[0:10]:
    URL = 'http://bioinfo.ibp.ac.cn/SmProt/' + SM_prot_code
    
    status, response = http.request(URL)
    ## Beaut Soup the response
    text = text_from_html(response)
    
elapsed = timeit.default_timer() - start_time
print(elapsed)

In [20]:
## Quick comparison between requests sessions and httplib2.http.request
## UPDATE - looks like requests.Session wins!
import timeit

results_per_page = 2000
orf_type = 'sORF'
gene_type = 'All'
function ='All'
organism = 'human'
datasource = 'All'

s = requests.Session()

start_time = timeit.default_timer()
for i in range(0,15):
    page_no = i 
    URL = "http://bioinfo.ibp.ac.cn/SmProt/browse.php?page=%s&perpage=%s&orfType=%s&geneType=%s&function=%s&org=%s&datasource=%s" %(page_no,results_per_page,orf_type,gene_type,function,organism,datasource)#
    response = s.get(URL)
    text = response.text
elapsed = timeit.default_timer() - start_time
print(elapsed)

10.21577886200248


In [25]:
## Quick comparison between requests sessions and httplib2.http.request
import timeit

results_per_page = 2000
orf_type = 'sORF'
gene_type = 'All'
function ='All'
organism = 'human'
datasource = 'All'

start_time = timeit.default_timer()
for i in range(0,15):
    page_no = i 
    URL = "http://bioinfo.ibp.ac.cn/SmProt/browse.php?page=%s&perpage=%s&orfType=%s&geneType=%s&function=%s&org=%s&datasource=%s" %(page_no,results_per_page,orf_type,gene_type,function,organism,datasource)#
    http = httplib2.Http()
    status, response = http.request(URL)
    
elapsed = timeit.default_timer() - start_time
print(elapsed)